In [2]:
import os
import json
import numpy as np
import matplotlib.pyplot as plt
print(os.getcwd())
##
import pandas as pd
# These lines in VSCode to avoid reloading the kernel when changes are made to the external kernel
%load_ext autoreload
%autoreload 2

/home/nicolas/documents/Master_Thesis_repo/OceanPlatformControl


In [3]:
import wandb
api = wandb.Api(timeout=60)

In [4]:
import re
def extract_number(string):
    pattern = re.compile(r"_(\d+)$")
    match = pattern.search(string)
    if match:
        return int(match.group(1))
    else:
        return None

In [5]:
experiment = "30_platforms_HC_HC"
ctrl = "flocking"
runs = api.runs(path="nhoischen/Master Thesis",
         filters={"$and": [{"config.experiment.id.value": experiment}, {"config.ctrlConfig.high_level_ctrl": ctrl}, 
         {"config.ctrlConfig.flocking_config.epsilon":1}]}
)
nb_missions = len(runs)
print(f"{nb_missions} missions for this experiment")

1191 missions for this experiment


In [6]:
metrics_dict = {}
run_ids = []
for nb_run, run in enumerate(runs):
    try:
        metrics_table = run.logged_artifacts()[0] # my metric table is logged as the first artifact object
        table_dir = metrics_table.download()
        table_path = f"{table_dir}/metrics.table.json"
        #metrics_url = run.summary['metrics']['path']
        metrics = json.load(open(table_path))
        for i, column in enumerate(metrics["columns"]):
            if nb_run == 0:
                metrics_dict[column] = [metrics["data"][0][i]]
            else:
                metrics_dict[column].append(metrics["data"][0][i])
        run_ids.append(extract_number(run.config['mission.id'])) # retrieve mission number, as for wandb dwnld the order between controllers don't match (allows to sort by mission id later)
    except BaseException:
        print(f"unable to find metrics for {run.id}")
        continue

df_metrics = pd.DataFrame(metrics_dict)

wandb:   1 of 1 files downloaded.  


unable to find metrics for 30_platforms_HC_HC_mission_nr_858_flocking_0_25_NoObserver


wandb:   1 of 1 files downloaded.  


unable to find metrics for 30_platforms_HC_HC_mission_nr_830_flocking_0_25_NoObserver


wandb:   1 of 1 files downloaded.  


unable to find metrics for 30_platforms_HC_HC_mission_nr_798_flocking_0_25_NoObserver
unable to find metrics for 30_platforms_HC_HC_mission_nr_6_flocking_0_25_NoObserver


wandb:   1 of 1 files downloaded.  


unable to find metrics for 30_platforms_HC_HC_mission_nr_695_flocking_0_25_NoObserver


wandb:   1 of 1 files downloaded.  


In [18]:
df_metrics['mission_nr'] = run_ids

In [19]:
df_metrics.to_csv(experiment + '_' + ctrl + '_' + str(len(run_ids)) + '.csv')

In [8]:
#df_sorted = df_metrics.sort_values(['mission_nr'])